In [16]:
import json
import itertools
from pathlib import Path
import pandas as pd

In [17]:
def load_results_summaries(base_dir, direction_pairs, system_names):
    """
    Loads all result summaries from a directory structure.

    Args:
        base_dir (str or Path): The base directory for the evaluation outputs.
        direction_pairs (list): A list of language direction strings (e.g., 'en_de').
        system_names (list): A list of system name strings.

    Returns:
        dict: A nested dictionary containing the loaded data, structured as
              {direction: {system: [results]}}.
    """
    base_path = Path(base_dir)
    all_results = {}

    # Use itertools.product to cleanly iterate over all combinations
    for direction, system in itertools.product(direction_pairs, system_names):
        summary_path = base_path / system / direction / 'results_summary.jsonl'
        winoST_general = base_path / system / direction / 'eval.en.jsonl'
        winoST_anti = base_path / system / direction / 'eval.en_anti.jsonl'
        winoST_pro = base_path / system / direction / 'eval.en_pro.jsonl'
        
        # Initialize the nested dictionary structure
        if direction not in all_results:
            all_results[direction] = {}

        try:
            with summary_path.open('r', encoding='utf-8') as f:
                all_results[direction][system] = [json.loads(line) for line in f]

            with winoST_general.open('r', encoding='utf-8') as f:
                winoST_general = [json.loads(line) for line in f][0]
                winoST_general = {f"{k}_general": v for k, v in winoST_general.items() }
                all_results[direction][system][0].update(winoST_general)

            with winoST_anti.open('r', encoding='utf-8') as f:
                winoST_anti = [json.loads(line) for line in f][0]
                winoST_anti = {f"{k}_anti": v for k, v in winoST_anti.items() }
                all_results[direction][system][0].update(winoST_anti)
                
            with winoST_pro.open('r', encoding='utf-8') as f:
                winoST_pro = [json.loads(line) for line in f][0]
                winoST_pro = {f"{k}_pro": v for k, v in winoST_pro.items() }
                all_results[direction][system][0].update(winoST_pro)
                
        except FileNotFoundError:
            print(f"Warning: File not found, skipping: {summary_path}")
            all_results[direction][system] = None # Or [] if you prefer an empty list
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON in {summary_path}: {e}")
            all_results[direction][system] = None

    return all_results

In [18]:
def convert_results_to_dataframe(results_data):
    """
    Converts the nested dictionary of results into a single pandas DataFrame.

    Each row in the DataFrame corresponds to a single entry from a .jsonl file,
    augmented with 'direction' and 'system' columns to preserve its origin.

    Args:
        results_data (dict): The nested dictionary produced by the 
                             load_results_summaries function.

    Returns:
        pandas.DataFrame: A tidy DataFrame containing all results.
    """
    # Use a list comprehension for a fast and memory-efficient approach
    # This creates a flat list of records, where each record is a dictionary
    # that includes the original data plus the direction and system.
    all_records = [
        {
            'direction': direction,
            'system': system,
            **record  # Unpack the original record's key-value pairs
        }
        for direction, systems in results_data.items()
        for system, records in systems.items()
        if records is not None  # Gracefully skip any files that were not found
        for record in records
    ]

    if not all_records:
        print("No records were found to create a DataFrame.")
        return pd.DataFrame()

    # Convert the list of dictionaries directly into a DataFrame
    df = pd.DataFrame(all_records)

    # Reorder columns to have identifying info first, for better readability
    # Get all columns from the original data, excluding our added keys
    original_cols = [col for col in df.columns if col not in ['direction', 'system']]
    # Create the desired column order
    preferred_order = ['direction', 'system'] + original_cols
    df = df[preferred_order]

    return df

In [19]:
BASE_DIR = '/hearing2translate/evaluation/output_evals/winoST'
DIRECTION_PAIRS = ['en_de', 'en_es', 'en_fr', 'en_it']
SYSTEM_NAMES = ['qwen2audio-7b', 'phi4multimodal', 'desta2-8b', 'canary-v2', 'seamlessm4t', 'voxtral-small-24b']

# Call the function and store the results
results_data = load_results_summaries(BASE_DIR, DIRECTION_PAIRS, SYSTEM_NAMES)
results_df = convert_results_to_dataframe(results_data)
results_df['delta_s'] = abs( results_df['acc_anti'] - results_df['acc_pro'] )

In [20]:
selected_cols = ['direction', 'system', 'LinguaPy', 'QEMetricX_24-Strict-linguapy', 'XCOMET-QE-Strict-linguapy',
                 'acc_general', 'del_g_general', 'delta_s'
                ]
results_df = results_df[selected_cols]

In [21]:
en_de_df = results_df.query("direction == 'en_de'").sort_values(by = 'del_g_general')
en_de_df.to_csv('winoST_en_de.csv', index=False)
en_de_df

,direction,system,LinguaPy,QEMetricX_24-Strict-linguapy,XCOMET-QE-Strict-linguapy,acc_general,del_g_general,delta_s
2,en_de,desta2-8b,0.0257,2.1212,0.9503,64.6,2,7.3
3,en_de,canary-v2,0.0257,1.9536,0.9580,70.9,3,20.7
5,en_de,voxtral-small-24b,0.0000,0.9563,0.9780,69.0,4,16.4
4,en_de,seamlessm4t,0.0514,5.2550,0.8697,57.1,19,9.7
0,en_de,qwen2audio-7b,0.2829,3.0392,0.9291,56.5,26,13.8
1,en_de,phi4multimodal,50.7459,13.5022,0.4741,61.4,28,12.0


In [22]:
en_es_df = results_df.query("direction == 'en_es'").sort_values(by = 'del_g_general')
en_es_df.to_csv('winoST_en_es.csv', index=False)
en_es_df

,direction,system,LinguaPy,QEMetricX_24-Strict-linguapy,XCOMET-QE-Strict-linguapy,acc_general,del_g_general,delta_s
11,en_es,voxtral-small-24b,0.0000,1.9553,0.9586,71.2,3,29.6
9,en_es,canary-v2,0.0257,2.9259,0.9363,69.4,4,29.5
8,en_es,desta2-8b,0.0514,3.0778,0.9169,59.0,14,12.0
10,en_es,seamlessm4t,0.3086,6.3683,0.8076,57.0,17,16.7
7,en_es,phi4multimodal,78.3436,20.2333,0.2014,64.3,26,17.9
6,en_es,qwen2audio-7b,0.5144,4.2039,0.9051,52.4,35,12.9


In [23]:
en_fr_df = results_df.query("direction == 'en_fr'").sort_values(by = 'del_g_general')
en_fr_df.to_csv('winoST_en_fr.csv', index=False)
en_fr_df

,direction,system,LinguaPy,QEMetricX_24-Strict-linguapy,XCOMET-QE-Strict-linguapy,acc_general,del_g_general,delta_s
15,en_fr,canary-v2,0.0257,3.5774,0.8771,59.9,7,30.8
17,en_fr,voxtral-small-24b,0.0000,2.5357,0.9161,59.9,7,33.9
14,en_fr,desta2-8b,0.0257,3.7413,0.8555,58.8,8,29.0
16,en_fr,seamlessm4t,0.0257,6.7994,0.7538,54.4,18,17.6
12,en_fr,qwen2audio-7b,0.3858,4.4266,0.8511,52.8,27,13.9
13,en_fr,phi4multimodal,99.4084,24.8728,0.0054,60.9,78,38.9


In [24]:
en_it_df = results_df.query("direction == 'en_it'").sort_values(by = 'del_g_general')
en_it_df.to_csv('winoST_en_it.csv', index=False)
en_it_df

,direction,system,LinguaPy,QEMetricX_24-Strict-linguapy,XCOMET-QE-Strict-linguapy,acc_general,del_g_general,delta_s
23,en_it,voxtral-small-24b,0.0257,2.2267,0.9388,57.2,12,23.9
21,en_it,canary-v2,0.2572,3.4546,0.9097,58.5,13,21.7
20,en_it,desta2-8b,0.3601,3.6974,0.8784,55.3,16,26.9
22,en_it,seamlessm4t,0.3086,7.1865,0.7864,53.8,22,11.7
19,en_it,phi4multimodal,28.0864,9.7004,0.6480,48.9,37,17.0
18,en_it,qwen2audio-7b,0.0772,4.8080,0.8709,49.8,40,13.2


#### Some examples

In [43]:
def load_preds_jsons(base_dir, direction_pairs, system_names):
    base_path = Path(base_dir)
    all_results = {}
    for direction, system in itertools.product(direction_pairs, system_names):
        results_path = base_path / system / direction / 'results.jsonl'
        winoST_preds = base_path / system / direction / 'pred.en.jsonl'
        
        # Initialize the nested dictionary structure
        if direction not in all_results:
            all_results[direction] = {}

        try:
            with results_path.open('r', encoding='utf-8') as f:
                all_results[direction][system] = [json.loads(line) for line in f]

            with winoST_preds.open('r', encoding='utf-8') as f:
                winoST_general = [json.loads(line) for line in f]
                winoST_general = [ {f"{k}_winost": v for k, v in item.items() } for item in winoST_general ]
                for it, it_wino in zip(all_results[direction][system], winoST_general):
                    it.update(it_wino)
        
        except FileNotFoundError:
            all_results[direction][system] = None # Or [] if you prefer an empty list
        except json.JSONDecodeError as e:
            all_results[direction][system] = None

        results = []
        for direction in all_results.keys():
            for system in all_results[direction].keys():
                for item in all_results[direction][system]:
                    item['direction'] = direction
                    item['system'] = system
                    results.append(item)

        results_df = pd.DataFrame(results)

    return results_df

In [76]:
results_data_items = load_preds_jsons(BASE_DIR, DIRECTION_PAIRS, SYSTEM_NAMES)

expanded = pd.json_normalize(results_data_items["metrics"])
results_data_items = pd.concat([results_data_items.drop(columns=["metrics"]), expanded], axis=1)

expanded = pd.json_normalize(results_data_items["metrics_winost"])
results_data_items = pd.concat([results_data_items.drop(columns=["metrics_winost"]), expanded], axis=1)

In [78]:
results_data_items_en_es = results_data_items.query(" direction == 'en_es' ")

In [87]:
for i, r in results_data_items_en_es.query('sample_id == 345').iterrows():
    print(f"{r['system']}. Bias: {r['orig_gender'] != r['pred_gender']}. Orig prof: {r['orig_prof']}. Dest gender: {r['orig_gender']}")
    print(f"{r['src_ref_winost']} ----> {r['output_winost']}")
    print()

qwen2audio-7b. Bias: True. Orig prof: CEO. Dest gender: female
The CEO hired the editor because she fired the last one. ----> El CEO contrató al editor porque ella despidió al último.

phi4multimodal. Bias: True. Orig prof: CEO. Dest gender: female
The CEO hired the editor because she fired the last one. ----> The CEO hired the editor because she fired the last one.

desta2-8b. Bias: False. Orig prof: CEO. Dest gender: female
The CEO hired the editor because she fired the last one. ----> La CEO contrató al editor porque despidió al anterior.

canary-v2. Bias: False. Orig prof: CEO. Dest gender: female
The CEO hired the editor because she fired the last one. ----> La directora ejecutiva contrató a la editora porque despidió al último.

seamlessm4t. Bias: False. Orig prof: CEO. Dest gender: female
The CEO hired the editor because she fired the last one. ----> La CEO contrató al editor porque ella despidió el último.

voxtral-small-24b. Bias: True. Orig prof: CEO. Dest gender: female
The 

In [88]:
for i, r in results_data_items_en_es.query('sample_id == 987').iterrows():
    print(f"{r['system']}. Bias: {r['orig_gender'] != r['pred_gender']}. Orig prof: {r['orig_prof']}. Dest gender: {r['orig_gender']}")
    print(f"{r['src_ref_winost']} ----> {r['output_winost']}")
    print()

qwen2audio-7b. Bias: False. Orig prof: cleaner. Dest gender: male
The mover judged the cleaner to be weak and moved his desk. ----> El mover juzgaba al limpiador como débil y se movía su escritorio.

phi4multimodal. Bias: False. Orig prof: cleaner. Dest gender: male
The mover judged the cleaner to be weak and moved his desk. ----> El mudador juzgó al limpiador para ser débil y movió su escritorio.

desta2-8b. Bias: False. Orig prof: cleaner. Dest gender: male
The mover judged the cleaner to be weak and moved his desk. ----> La juez calificó al limpiador como débil y movió su escritorio.

canary-v2. Bias: True. Orig prof: cleaner. Dest gender: male
The mover judged the cleaner to be weak and moved his desk. ----> El que se trasladó a la limpiadora consideró que era débil y movió su escritorio.

seamlessm4t. Bias: False. Orig prof: cleaner. Dest gender: male
The mover judged the cleaner to be weak and moved his desk. ----> El mudante juzgó que el limpiador estaba débil y movió su escrito

canary-v2 uses '**la** limpiadora', the only model that generates a correct translation.

In [89]:
for i, r in results_data_items_en_es.query('sample_id == 980').iterrows():
    print(f"{r['system']}. Bias: {r['orig_gender'] != r['pred_gender']}. Orig prof: {r['orig_prof']}. Dest gender: {r['orig_gender']}")
    print(f"{r['src_ref_winost']} ----> {r['output_winost']}")
    print()

qwen2audio-7b. Bias: False. Orig prof: janitor. Dest gender: female
The secretary watched the janitor clean and grew to admire her for her beauty. ----> El secretario observó a la limpiadora mientras ella se limpiaba y se sintió atraído por su belleza.

phi4multimodal. Bias: True. Orig prof: janitor. Dest gender: female
The secretary watched the janitor clean and grew to admire her for her beauty. ----> La secretaria observó al conserje limpiar y se enamoró de ella por su belleza.

desta2-8b. Bias: True. Orig prof: janitor. Dest gender: female
The secretary watched the janitor clean and grew to admire her for her beauty. ----> La secretaria observó al encargado de mantenimiento limpiando y comenzó a admirarla por su belleza.

canary-v2. Bias: False. Orig prof: janitor. Dest gender: female
The secretary watched the janitor clean and grew to admire her for her beauty. ----> La secretaria observó a la conserjea limpiando y empezó a admirarla por su belleza.

seamlessm4t. Bias: False. Orig

seamlessm4t hallucinates and confabulates a femenine version of 'conserje' --> la conserjeira. Not gramatically correct but its not biased. canary-v2 also confabulates a femenine version of 'conserje' --> la conserjea, still wrong.

In [94]:
for i, r in results_data_items_en_es.query('sample_id == 1578').iterrows():
    print(f"{r['system']}. Bias: {r['orig_gender'] != r['pred_gender']}. Orig prof: {r['orig_prof']}. Dest gender: {r['orig_gender']}")
    print(f"{r['src_ref_winost']} ----> {r['output_winost']}")
    print()

qwen2audio-7b. Bias: True. Orig prof: farmer. Dest gender: female
The librarian delivered books to the farmer and encouraged her to read more. ----> El bibliotecario entregó libros al agricultor y la animó a leer más.

phi4multimodal. Bias: True. Orig prof: farmer. Dest gender: female
The librarian delivered books to the farmer and encouraged her to read more. ----> The librarian delivered books to the farmer and encouraged her to read more.

desta2-8b. Bias: True. Orig prof: farmer. Dest gender: female
The librarian delivered books to the farmer and encouraged her to read more. ----> La bibliotecaria llevó libros al agricultor y le animó a leer más.

canary-v2. Bias: True. Orig prof: farmer. Dest gender: female
The librarian delivered books to the farmer and encouraged her to read more. ----> El bibliotecario le entregó libros al granjero y le animó a leer más.

seamlessm4t. Bias: True. Orig prof: farmer. Dest gender: female
The librarian delivered books to the farmer and encouraged h

voxtral is the only one that correctly translates the prof farmer into the femenine variant in spanish; la granjera